In [1]:
!pip install transformers clean-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 5.8 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171034 sha256=597eff9dca9673f1b025ee7d631af89fe8da9431f23233edcc298de9061936e7
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji


In [2]:
from transformers import DistilBertTokenizer, DistilBertConfig, TFDistilBertModel
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tqdm import tqdm
from cleantext import clean

In [4]:
import numpy as np
import pandas as pd


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
df_train = pd.read_csv('drive/MyDrive/Bert_learn/train.csv')
df_val = pd.read_csv('drive/MyDrive/Bert_learn/valid.csv')

In [7]:
# X_train = list(df_train['text'])
# y_train = list(df_train['label'])

# X_test = list(df_val['text'])
# y_test = list(df_val['label'])

In [3]:
distil_bert = 'distilbert-base-uncased'

tokenizer = DistilBertTokenizer.from_pretrained(distil_bert, do_lower_case=True, add_special_tokens=True,
                                                max_length=128, pad_to_max_length=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [8]:
def cleaning(text):
    return clean(text, no_line_breaks=True, no_urls=True, no_punct=True)

def tokenize(sentences, tokenizer):

    input_ids = []
    input_masks = []

    for sentence in tqdm(sentences):
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True,
                                       max_length=128, pad_to_max_length=True,
                                       return_attention_mask=True, return_token_type_ids=True)

        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])

    return np.asarray(input_ids, dtype='int32'),np.asarray(input_masks, dtype='int32')

In [14]:
df_train['text'] = df_train['text'].apply(cleaning)
train_input_ids, train_input_masks = tokenize(df_train['text'], tokenizer)

df_val['text'] = df_val['text'].apply(cleaning)
val_input_ids, val_input_masks = tokenize(df_val['text'], tokenizer)

  0%|          | 0/40000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
  0%|          | 0/5000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length

In [ ]:
# X_train = list(df_train['text'])
# y_train = list(df_train['label'])

# X_test = list(df_val['text'])
# y_test = list(df_val['label'])

In [26]:
config = DistilBertConfig(dropout=0.2, attention_dropout=0.2)

config.output_hidden_states = False

transformer_model = TFDistilBertModel.from_pretrained(distil_bert, config=config)

input_ids_in = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
input_masks_in = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32')

embedding_layer = transformer_model(input_ids_in, attention_mask=input_masks_in)[0]
X = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,
                                                       return_sequences=True,
                                                       dropout=0.1,
                                                       recurrent_dropout=0.1))(embedding_layer)
X = tf.keras.layers.GlobalMaxPool1D()(X)
X = tf.keras.layers.Dense(50, activation='relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)
X = tf.keras.layers.Dense(1, activation='sigmoid')(X)

model = tf.keras.models.Model(inputs=[input_ids_in, input_masks_in], outputs=X)

for layer in model.layers[:3]:
    layer.trainable = False

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [27]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_token (InputLayer)    [(None, 128)]                0         []                            
                                                                                                  
 masked_token (InputLayer)   [(None, 128)]                0         []                            
                                                                                                  
 tf_distil_bert_model_1 (TF  TFBaseModelOutput(last_hid   6636288   ['input_token[0][0]',         
 DistilBertModel)            den_state=(None, 128, 768)   0          'masked_token[0][0]']        
                             , hidden_states=None, atte                                           
                             ntions=None)                                                   

In [28]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
# X_train_id, X_test_id, X_train_mask, X_test_mask, y_train, y_test = train_test_split(input_ids,
#                                                                                      input_masks,
#                                                                                      y_labels,
#                                                                                      test_size=0.2,
#                                                                                      random_state=42)


X_train_id = train_input_ids
X_test_id = val_input_ids
X_train_mask = train_input_masks
X_test_mask = val_input_masks
y_train = df_train['label']
y_test = df_val['label']


In [23]:
X_train_id.shape,X_train_mask.shape,y_train.shape

((40000, 128), (40000, 128), (40000,))

In [20]:
X_test_id.shape, X_test_mask.shape

((5000, 128), (5000, 128))

In [ ]:
hist = model.fit([X_train_id, X_train_mask],
                 y_train,
                 validation_data=([X_test_id, X_test_mask], y_test),
                 epochs=1,
                 batch_size=64)

/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(64, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


 36/625 [>.............................] - ETA: 3:16:00 - loss: 0.0000e+00 - accuracy: 0.4970

In [ ]:
model.save_weights('toxic.h5')

In [ ]:
sample_text = 'I hate you, you idiot!'
clean_txt = cleaning(sample_text)
input_ids_test, input_masks_test = tokenize(clean_txt, tokenizer)